In [1]:
"""
교차검증 : Cross Validation
"""

'\n교차검증 : Cross Validation\n'

In [2]:
import pandas as pd
import numpy as np

In [3]:
### 데이터 읽어들이기
# 08_wine.csv
# 데이터프레임 변수명 : wine
file_path = "./data/08_wine.csv"
wine = pd.read_csv(file_path)
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
### 입력(독립변수) 및 타겟(종속변수) 데이터 추출하기
# 입력변수 데이터 변수 :  data
# 종속변수 데이터 변수 : target
# numpy의 배열 함수 사용

data = wine[["alcohol", "sugar", "pH"]].to_numpy()   # 행렬형태를 배열형태로 변환 to_numpy()
target = wine["class"].to_numpy()

data.shape, target.shape

((6497, 3), (6497,))

In [5]:
### 훈련/검증/테스트 데이터로 분류하기
"""
- 일반적으로 테스트 데이터는 마지막에 딱 한번만 실시함
- 이런 경우, 훈련모델 성능 향상(과적합여부 등)등에 사용할 데이터를 추가로 분리
- 추가로 분리하는 데이터를 검증데이터라고 보통 칭함
- 훈련 : 검증 : 테스트 = 6 : 2 : 2 정도
                       = 5 : 2 : 3 정도
- 최근에는 학술연구 또는 연구과제 시에 검증데이터를 분류해 했는지 여부 확인                       
"""

'\n- 일반적으로 테스트 데이터는 마지막에 딱 한번만 실시함\n- 이런 경우, 훈련모델 성능 향상(과적합여부 등)등에 사용할 데이터를 추가로 분리\n- 추가로 분리하는 데이터를 검증데이터라고 보통 칭함\n- 훈련 : 검증 : 테스트 = 6 : 2 : 2 정도\n                       = 5 : 2 : 3 정도\n- 최근에는 학술연구 또는 연구과제 시에 검증데이터를 분류해 했는지 여부 확인                       \n'

In [6]:
### 훈련 : 검증 : 테스트 = 6 : 2 : 2
# 1. 훈련과 테스트로 먼저 분류 : 8 : 2
# 2. 훈련과 검증으로 분류 : 6 : 2

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# 1. 훈련과 테스트로 먼저 분류 : 8 : 2
train_input, test_input, train_target, test_target = \
              train_test_split(data, target, test_size= 0.2, random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [9]:
### 2. 훈련과 검증으로 분류 : 6 : 2
sub_input, val_input, sub_target, val_target = \
              train_test_split(train_input, train_target, test_size= 0.2, random_state=42)

print(sub_input.shape, sub_target.shape)
print(val_input.shape, val_target.shape)
print(test_input.shape, test_target.shape)

(4157, 3) (4157,)
(1040, 3) (1040,)
(1300, 3) (1300,)


# 결정트리 모델로 훈련시켜보기

In [10]:
### 결정트리 모델로 훈련, 검증, 테스트 정확도 확인하기

In [11]:
### 라이브러리 불러들이기
from sklearn.tree import DecisionTreeClassifier

In [12]:
### 정규화하기
# - train_scaled, test_scaled

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(sub_input)

sub_input_scaled = ss.transform(sub_input)
val_input_scaled = ss.transform(val_input)
test_input_scaled = ss.transform(test_input)
sub_input_scaled.shape, val_input_scaled.shape, test_input_scaled.shape

((4157, 3), (1040, 3), (1300, 3))

In [13]:
### 모델(클래스) 생성하기
dt = DecisionTreeClassifier(random_state=42)
dt

DecisionTreeClassifier(random_state=42)

In [14]:
### 훈련시키기
dt.fit(sub_input_scaled, sub_target)

DecisionTreeClassifier(random_state=42)

In [15]:
### test 정확도는 최종 dt 모델의 최적화가 긑난 후 한번만 실행합니다.
# - 최적화 : 과대/과소 적합 해소 후 최종 모델이 완료된 상태

### 평가하기(정확도)
train_score = dt.score(sub_input_scaled, sub_target)
val_score = dt.score(val_input_scaled, val_target)
test_score = dt.score(test_input_scaled, test_target)
train_score, val_score, test_score

### 훈련정확도 : 과대적합 발생
# 검증과 테스트 정확도는 비슷함(과대/과소 없음)

(0.9971133028626413, 0.8625, 0.8561538461538462)

# 교차검증

In [16]:
"""
- 훈련과 검증으로 데이터를 구분할 필요 없음
- 내부적으로 훈련 및 검증 데이터로 구분하여 사용함(사람이 관여 안함)
- 데이터 준비는 기존처럼 훈련 및 테스트 데이터만 있으면 됨.
"""

'\n- 훈련과 검증으로 데이터를 구분할 필요 없음\n- 내부적으로 훈련 및 검증 데이터로 구분하여 사용함(사람이 관여 안함)\n- 데이터 준비는 기존처럼 훈련 및 테스트 데이터만 있으면 됨.\n'

In [17]:
### 교차검증 라이브러리
from sklearn.model_selection import cross_validate

In [18]:
### 교차검증 시키기
# 첫번째 : 사용한 훈련 모델
# 두번째 : 훈련 독립변수
# 세번째 : 훈련 종속변수
score = cross_validate(dt, train_input, train_target)
score

### 해석
# - fit_time : 훈련에 걸린 시간들
# - score_tiem : 검증에 걸린 시간들
# - test_score : 검증 적확도들
# ** 최종 정확도는 test_score의 평균값이 사용됨

{'fit_time': array([0.        , 0.0143826 , 0.00134397, 0.        , 0.        ]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [19]:
### 훈련 검증결과
# test_score의 평균을 사용
np.mean(score["test_score"])

0.855300214703487

In [20]:
"""
< 교차검증에 사용되는 주요 키워드 >
 - k-폴드(k-fold) 교차검증 : 훈련데이터를 k개의 구간으로 나눠서 훈련 수행
 - 각 구간마다 훈련데이터를 검증데이터로 바꿔가면서 훈련 시킴
 - 보텅 5-폴드 또는 10-폴드 교차검증을 주로 사용함
 - 훈련 : 테스트 비율은 8:2를 주로 사용함
 - k-폴드 또는 k-겹 교차검증이라는 용어를 사용함
"""

'\n< 교차검증에 사용되는 주요 키워드 >\n - k-폴드(k-fold) 교차검증 : 훈련데이터를 k개의 구간으로 나눠서 훈련 수행\n - 각 구간마다 훈련데이터를 검증데이터로 바꿔가면서 훈련 시킴\n - 보텅 5-폴드 또는 10-폴드 교차검증을 주로 사용함\n - 훈련 : 테스트 비율은 8:2를 주로 사용함\n - k-폴드 또는 k-겹 교차검증이라는 용어를 사용함\n'

In [21]:
"""
< 교차검증 구간 예시 :  3-폴드 라고 가정했을 때 >
 - 훈련 독립변수 데이터와 훈련 종속변수 데이터를 3구간으로 나눔
 - 데이터의 기준 위치(검증데이터 = 종속변수 데이터를 의미함)
   훈련데이터(1위치) | 훈련데이터(2위치) | 훈련데이터(3위치)
   독립데이터(1위치) | 독립데이터(2위치) | 독립데이터(3위치)
   
   (1폴드) 훈련(1위치) | 훈련(2위치) | 검증(3위치)
      - 훈련데이터(1위치 + 2위치) | 검증데이터(3위치)
   (2폴드) 훈련(1위치) | 검증(2위치) | 훈련(3위치)
      - 훈련데이터(1위치 + 3위치) | 검증데이터(2위치)
   (3폴드) 검증(1위치) | 훈련(2위치) | 훈련(3위치)
      - 훈련데이터(2위치 + 3위치) | 검증데이터(1위치)
      
    최종 검증 결과 : 각 폴드의 검증 결과를 평균한 값을 사용
    ** 훈련과 검증을 더 쪼개서 훈련을 시키기 때문에 안정된 평가 결과 얻음
"""

'\n< 교차검증 구간 예시 :  3-폴드 라고 가정했을 때 >\n - 훈련 독립변수 데이터와 훈련 종속변수 데이터를 3구간으로 나눔\n - 데이터의 기준 위치(검증데이터 = 종속변수 데이터를 의미함)\n   훈련데이터(1위치) | 훈련데이터(2위치) | 훈련데이터(3위치)\n   독립데이터(1위치) | 독립데이터(2위치) | 독립데이터(3위치)\n   \n   (1폴드) 훈련(1위치) | 훈련(2위치) | 검증(3위치)\n      - 훈련데이터(1위치 + 2위치) | 검증데이터(3위치)\n   (2폴드) 훈련(1위치) | 검증(2위치) | 훈련(3위치)\n      - 훈련데이터(1위치 + 3위치) | 검증데이터(2위치)\n   (3폴드) 검증(1위치) | 훈련(2위치) | 훈련(3위치)\n      - 훈련데이터(2위치 + 3위치) | 검증데이터(1위치)\n      \n    최종 검증 결과 : 각 폴드의 검증 결과를 평균한 값을 사용\n    ** 훈련과 검증을 더 쪼개서 훈련을 시키기 때문에 안정된 평가 결과 얻음\n'

# 폴더(fold) 클래스 사용하기

In [22]:
### 폴드(fold) 컨트롤을 위한 라이브러리
# - 훈련데이터를 섞거나, 폴드의 갯수 지정 가능
from sklearn.model_selection import StratifiedKFold 

In [23]:
### 폴드 클래스 사용하기
# - cv :  분할기 라고 칭합니다.
#      : 분할기로 사용할 클래스 지정
#      : 클래스 속성값이 없을 경우 , 기본 5폴드 사용됨
#                                  , 기본 섞지 않음
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(scores)
print("훈련정확도 : ", np.mean(scores["test_score"]))

{'fit_time': array([0.        , 0.        , 0.00136065, 0.        , 0.        ]), 'score_time': array([0.        , 0.01440597, 0.        , 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
훈련정확도 :  0.855300214703487


In [24]:
### 폴드 갯수 지정 및 데이터 섞기
# n_splits :  폴드 갯수
# shuffle : True(섞기), Fallse(안섞기)

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(scores)
print("훈련정확도 : ", np.mean(scores["test_score"]))

{'fit_time': array([0.        , 0.01415181, 0.        , 0.        , 0.01656628,
       0.00398827, 0.00394845, 0.00398684, 0.00498343, 0.00498295]), 'score_time': array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00099587, 0.00099659, 0.00099683, 0.        , 0.00099707]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}
훈련정확도 :  0.8574181117533719


# 하이퍼파라메터 튜닝(AutoML)

In [25]:
"""
< AutoML >
 - 교차검증, 하이퍼파라메터 찾기, 모델훈련을 한번에 자동으로 수행
 - 사용패키지 : sklearn.model_selection
 - 사용클래스 : GridSearchCV
"""

'\n< AutoML >\n - 교차검증, 하이퍼파라메터 찾기, 모델훈련을 한번에 자동으로 수행\n - 사용패키지 : sklearn.model_selection\n - 사용클래스 : GridSearchCV\n'

In [26]:
### max_dept(트리의 깊이) 값 찾기
from sklearn.model_selection import GridSearchCV 

In [27]:
### 찾을 하이퍼파라메터 속성 및 값(범위) 설정
# - 딕셔너리로 생성해야함
# - 찾을 하이퍼파라메터 key값이 여러개인 경우 콤마(,)로 구분하여 추가
params = {"max_depth" : range(5, 20, 1)}
params

{'max_depth': range(5, 20)}

In [28]:
### AutoML 클래스 : 
# - 첫번째 값 : 어떤 훈련모델을 사용할지 지정
# - 두번째 값 :  찾을 하이퍼라파메터 지정
# - 세번째 값 :  사용할 cpu 코어 갯수 지정 (-1은 전체 core 사용)
#             : 기본값은 1

# - 훈련반복 횟수 : 기본폴드 5 * max_dept범위 15개 = 75회 훈련 반복됨
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), param_grid=params, n_jobs = -1)
gs

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [29]:
### 훈련 시키기
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [30]:
### 자동으로 찾은 하이퍼파라메터 확인하기
# - 확인하는 속성명 : best_estimator_
# - 확인시 중요사항 : 결정트리 훈련모델로 사용할 경우에는
# --> best_estimator_의 객체를 그대로 받아서 사용해야함
gs.best_estimator_

DecisionTreeClassifier(max_depth=8, random_state=42)

In [31]:
### 훈련 정확도 확인을 위해 best_estimator_ 속성값 받아서 사용
dt = gs.best_estimator_
dt

DecisionTreeClassifier(max_depth=8, random_state=42)

In [32]:
### 훈련 정확도 확인하기
dt.score(train_input, train_target)

0.9003271117952665

In [33]:
### 찾고자 했던 max_depth 값 확인하기
gs.best_params_

{'max_depth': 8}

In [34]:
### 검증 정확도 확인하기 (기본폴ㄷ으 5에 대한 평균값)
gs.best_score_

0.859729029392167

In [35]:
### AutoML 의 모든 수행결과 확인하기
gs.cv_results_

{'mean_fit_time': array([0.00308733, 0.0045845 , 0.00547752, 0.00453629, 0.00373201,
        0.0084487 , 0.00578051, 0.00617933, 0.00657797, 0.00617905,
        0.00657773, 0.00637789, 0.00617895, 0.00637827, 0.00366364]),
 'std_fit_time': array([0.00155619, 0.00048877, 0.00600613, 0.00655985, 0.00215142,
        0.00469727, 0.0003989 , 0.00074591, 0.00048803, 0.0003983 ,
        0.00048813, 0.00048836, 0.00039883, 0.00048873, 0.00227222]),
 'mean_score_time': array([0.00260029, 0.00099716, 0.00039873, 0.00341558, 0.00059824,
        0.00109439, 0.00099678, 0.00099664, 0.00079737, 0.00099659,
        0.00079746, 0.00099721, 0.00079722, 0.00093241, 0.        ]),
 'std_score_time': array([2.96880140e-03, 8.34124359e-07, 4.88344054e-04, 5.85146241e-03,
        4.88460855e-04, 1.95241229e-04, 6.46813391e-07, 2.78041453e-07,
        3.98683562e-04, 2.61174468e-07, 7.46212849e-04, 4.67203091e-07,
        3.98612033e-04, 1.29081008e-04, 0.00000000e+00]),
 'param_max_depth': masked_array(data=

In [36]:
gs.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [37]:
### params의 값 확인하기
gs.cv_results_["params"]

[{'max_depth': 5},
 {'max_depth': 6},
 {'max_depth': 7},
 {'max_depth': 8},
 {'max_depth': 9},
 {'max_depth': 10},
 {'max_depth': 11},
 {'max_depth': 12},
 {'max_depth': 13},
 {'max_depth': 14},
 {'max_depth': 15},
 {'max_depth': 16},
 {'max_depth': 17},
 {'max_depth': 18},
 {'max_depth': 19}]

In [38]:
### 1폴드의 검증결과 확인하기
# - max_depth의 범위 갯수만큼 나옴(15개)
gs.cv_results_["split0_test_score"]

array([0.84711538, 0.84807692, 0.85769231, 0.85288462, 0.85769231,
       0.84519231, 0.85865385, 0.86730769, 0.86538462, 0.86826923,
       0.86730769, 0.86442308, 0.86346154, 0.86634615, 0.87211538])

In [39]:
### max_depth가 8번째일때의 값
gs.best_score_

0.859729029392167

In [40]:
### 교차 검증값 조회
gs.cv_results_["mean_test_score"]
np.mean(gs.cv_results_["mean_test_score"])

0.85722551269712

In [41]:
### 훈련정확도 : 테스트정확도
gs.best_score_, dt.score(test_input, test_target)

(0.859729029392167, 0.8584615384615385)